In [28]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine

In [29]:
engine = create_engine('postgresql://localhost:5432/ETL Project')
connection = engine.connect()

OperationalError: (psycopg2.OperationalError) fe_sendauth: no password supplied

(Background on this error at: http://sqlalche.me/e/e3q8)

In [2]:
TSLA_data = pd.read_csv("TSLA.csv")
Tweet_data = pd.read_csv("data_elonmusk_before.csv", encoding='iso-8859-1')

In [3]:
Tweet_data.head()


,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,9/29/2017 17:39,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,9/29/2017 10:44,SpaceX,elonmusk
2,Row2,@bigajm Yup :),9/29/2017 10:39,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,9/29/2017 9:56,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,9/29/2017 9:19,NaN,elonmusk


In [4]:
TSLA_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,35.000000,35.450001,34.709999,35.360001,35.360001,1194800
1,2013-01-03,35.180000,35.450001,34.750000,34.770000,34.770000,742000
2,2013-01-04,34.799999,34.799999,33.919998,34.400002,34.400002,674000
3,2013-01-07,34.799999,34.799999,33.900002,34.340000,34.340000,442000
4,2013-01-08,34.500000,34.500000,33.110001,33.680000,33.680000,1284000


In [5]:
Tweet_data["Retweet from"].fillna("Not Retweeted", inplace = True)
Tweet_data.head(20)

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,9/29/2017 17:39,Not Retweeted,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,9/29/2017 10:44,SpaceX,elonmusk
2,Row2,@bigajm Yup :),9/29/2017 10:39,Not Retweeted,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,9/29/2017 9:56,Not Retweeted,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,9/29/2017 9:19,Not Retweeted,elonmusk
5,Row5,RT @SpaceX: Supporting the creation of a perma...,9/29/2017 8:57,SpaceX,elonmusk
6,Row6,BFR will take you anywhere on Earth in less th...,9/29/2017 8:53,Not Retweeted,elonmusk
7,Row7,Mars City\nOpposite of Earth. Dawn and dusk sk...,9/29/2017 6:03,Not Retweeted,elonmusk
8,Row8,Moon Base Alpha https://t.co/voY8qEW9kl,9/29/2017 5:44,Not Retweeted,elonmusk
9,Row9,Will be announcing something really special at...,9/29/2017 2:36,Not Retweeted,elonmusk


In [6]:
TSLA_data.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [7]:
Tweet_data.dtypes


row ID          object
Tweet           object
Time            object
Retweet from    object
User            object
dtype: object

In [8]:
Tweet_data['Date'] = pd.to_datetime(Tweet_data['Time']).dt.date

In [9]:
Tweet_data.head(30)

,row ID,Tweet,Time,Retweet from,User,Date
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,9/29/2017 17:39,Not Retweeted,elonmusk,2017-09-29
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,9/29/2017 10:44,SpaceX,elonmusk,2017-09-29
2,Row2,@bigajm Yup :),9/29/2017 10:39,Not Retweeted,elonmusk,2017-09-29
3,Row3,Part 2 https://t.co/8Fvu57muhM,9/29/2017 9:56,Not Retweeted,elonmusk,2017-09-29
4,Row4,Fly to most places on Earth in under 30 mins a...,9/29/2017 9:19,Not Retweeted,elonmusk,2017-09-29
5,Row5,RT @SpaceX: Supporting the creation of a perma...,9/29/2017 8:57,SpaceX,elonmusk,2017-09-29
6,Row6,BFR will take you anywhere on Earth in less th...,9/29/2017 8:53,Not Retweeted,elonmusk,2017-09-29
7,Row7,Mars City\nOpposite of Earth. Dawn and dusk sk...,9/29/2017 6:03,Not Retweeted,elonmusk,2017-09-29
8,Row8,Moon Base Alpha https://t.co/voY8qEW9kl,9/29/2017 5:44,Not Retweeted,elonmusk,2017-09-29
9,Row9,Will be announcing something really special at...,9/29/2017 2:36,Not Retweeted,elonmusk,2017-09-29


In [10]:
Tweet_data.dtypes

row ID          object
Tweet           object
Time            object
Retweet from    object
User            object
Date            object
dtype: object

In [11]:
clean_tweet_data = Tweet_data[["Tweet", "Retweet from", "User", "Date"]]

In [12]:
clean_tweet_data.head()

,Tweet,Retweet from,User,Date
0,@MeltingIce Assuming max acceleration of 2 to ...,Not Retweeted,elonmusk,2017-09-29
1,RT @SpaceX: BFR is capable of transporting sat...,SpaceX,elonmusk,2017-09-29
2,@bigajm Yup :),Not Retweeted,elonmusk,2017-09-29
3,Part 2 https://t.co/8Fvu57muhM,Not Retweeted,elonmusk,2017-09-29
4,Fly to most places on Earth in under 30 mins a...,Not Retweeted,elonmusk,2017-09-29


In [14]:
clean_tweet_data["Date"] = pd.to_datetime(clean_tweet_data["Date"])



Tweet                   object
Retweet from            object
User                    object
Date            datetime64[ns]
dtype: object

In [15]:
clean_tweet_data.dtypes

Tweet                   object
Retweet from            object
User                    object
Date            datetime64[ns]
dtype: object

In [19]:
TSLA_data["Date"] = pd.to_datetime(TSLA_data["Date"])

In [20]:
TSLA_data.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object